# Lesson 3: RAG with Web Access

In this lesson, we'll learn how to create an AI agent that can access the web in real-time using the Brave Search API. This approach allows our LLM to provide up-to-date information and answer queries about current events or topics that may not be in its training data.

## Lesson Objectives

By the end of this lesson, you will be able to:

1. Make API calls to Brave Search to retrieve web information
2. Process and extract relevant information from search results
3. Integrate web search capabilities with LLM using tools and functions
4. Create a conversational agent that can answer questions using real-time web data
5. Understand best practices for prompting and chain-of-thought reasoning in web-enabled agents

## 1. Environment Setup

To get started, we'll need to set up our environment by installing the necessary libraries and configuring our API keys.

In [ ]:
# Check and install dependencies if needed
# Uncomment and run if required

# !pip install requests
# !pip install python-dotenv
# !pip install openai

In [39]:
# Import required libraries
import requests
import os
import json
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI
from IPython.display import display, HTML, Markdown

# Load environment variables
load_dotenv(find_dotenv())

True

In [40]:
# Get the Keys
API_KEY = os.environ.get("AZURE_OPENAI_KEY") 
API_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
API_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
MODEL = os.environ.get("AZURE_OPENAI_MODEL")
BRAVE_SEARCH_API_KEY = os.environ.get("BRAVE_SEARCH_API_KEY")

# Check if the necessary API keys are available
if not API_KEY:
    print("⚠️ Brave Search API key not found. Please set the BRAVE_SEARCH_API_KEY environment variable.")
if not BRAVE_SEARCH_API_KEY:
    print("⚠️ OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

In [41]:
client = AzureOpenAI(
  default_headers={"Ocp-Apim-Subscription-Key": API_KEY},
  api_key=API_KEY,
  azure_endpoint=API_ENDPOINT,
  azure_deployment= AZURE_DEPLOYMENT,
  api_version=API_VERSION, 
)

## 2. Brave Search API Basics

The Brave Search API allows us to programmatically search the web and retrieve results. Before we integrate it with an LLM, let's understand how to use it directly.

The example below is similar to the code in `example_3.py` and shows a simple query to find Greek restaurants in San Francisco.

In [5]:
def brave_search(query, count=10):
    """
    Perform a web search using the Brave Search API
    
    Args:
        query: The search query
        count: Number of results to return (default: 10)
    
    Returns:
        dict: JSON response from the API
    """
    url = "https://api.search.brave.com/res/v1/web/search"

    response = requests.get(url, headers={"X-Subscription-Token": BRAVE_SEARCH_API_KEY},
                            params={
                                "q": query,
                                "count": count,
                                "country": "us",
                                "search_lang": "en",
                            },
                        )
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None
    
    return response.json()

In [10]:
# Example search for Greek restaurants in San Francisco
results = brave_search("greek restaurants in san francisco")

In [ ]:
# check the type of results
type(results)

dict

In [12]:
results

{'query': {'original': 'greek restaurants in san francisco',
  'show_strict_warning': False,
  'is_navigational': True,
  'is_geolocal': True,
  'local_decision': 'mix',
  'local_locations_idx': 1,
  'is_news_breaking': False,
  'spellcheck_off': True,
  'country': 'us',
  'bad_results': False,
  'should_fallback': False,
  'postal_code': '',
  'city': '',
  'header_country': '',
  'more_results_available': True,
  'state': ''},
 'mixed': {'type': 'mixed',
  'main': [{'type': 'web', 'index': 0, 'all': False},
   {'type': 'web', 'index': 1, 'all': False},
   {'type': 'web', 'index': 2, 'all': False},
   {'type': 'web', 'index': 3, 'all': False},
   {'type': 'web', 'index': 4, 'all': False},
   {'type': 'web', 'index': 5, 'all': False},
   {'type': 'web', 'index': 6, 'all': False},
   {'type': 'web', 'index': 7, 'all': False},
   {'type': 'web', 'index': 8, 'all': False},
   {'type': 'web', 'index': 9, 'all': False},
   {'type': 'web', 'index': 10, 'all': False},
   {'type': 'web', 'inde

In [18]:
type(results.get("web"))

dict

In [19]:
type(results.get("web").get("results"))

list

In [ ]:
results.get("web").get("results") # output is a list of dictionaries, each representing a search result

[{'title': 'Kokkari Estiatorio Home Page - KOKKARI',
  'url': 'https://kokkari.com/',
  'is_source_local': False,
  'is_source_both': False,
  'description': 'Philoxenia Philoxenia The art of making a stranger a friend In the heart of the <strong>San</strong> <strong>Francisco</strong> Financial District you will find',
  'page_age': '2025-06-05T00:00:00',
  'profile': {'name': 'KOKKARI',
   'url': 'https://kokkari.com/',
   'long_name': 'kokkari.com',
   'img': 'https://imgs.search.brave.com/hWC1bPorpFs6XKZa6tL6VJw1YxSdFuJNkMUguOdjxJc/rs:fit:32:32:1:0/g:ce/aHR0cDovL2Zhdmlj/b25zLnNlYXJjaC5i/cmF2ZS5jb20vaWNv/bnMvNTVjZDliZDYy/ZDhlZGI0MGYwMGQ5/NzkyZjcxZTU1ZWFm/ZDI5YTBlMGEyZWMz/N2VkMjVmNmE4Mzlk/MjY3ZDAxNS9rb2tr/YXJpLmNvbS8'},
  'language': 'en',
  'family_friendly': True,
  'type': 'search_result',
  'subtype': 'generic',
  'is_live': False,
  'meta_url': {'scheme': 'https',
   'netloc': 'kokkari.com',
   'hostname': 'kokkari.com',
   'favicon': 'https://imgs.search.brave.com/hWC1bPorpFs6X

In [34]:
# Let's print a more readable version of the results
if results:
    print(f"Total results found: {len(results.get('web', {}).get('results', []))}") # {}, [], handle cases where 'web' or 'results' might not exist
    
    # Print the first 3 results in a readable format
    no_results = 3
    for i, result in enumerate(results.get('web', {}).get('results', [])[:no_results]):
        print(f"\n--- Result {i+1} ---")
        print(f"Title: {result.get('title', 'No title')}")
        print(f"URL: {result.get('url', 'No URL')}")
        print(f"Description: {result.get('description', 'No description')[:200]}...")

Total results found: 10

--- Result 1 ---
Title: Kokkari Estiatorio Home Page - KOKKARI
URL: https://kokkari.com/
Description: Philoxenia Philoxenia The art of making a stranger a friend In the heart of the <strong>San</strong> <strong>Francisco</strong> Financial District you will find...

--- Result 2 ---
Title: THE 10 BEST Greek Restaurants in San Francisco (Updated 2025)
URL: https://www.tripadvisor.com/Restaurants-g60713-c23-San_Francisco_California.html
Description: Best <strong>Greek</strong> <strong>Restaurants</strong> <strong>in</strong> <strong>San</strong> <strong>Francisco</strong>, California: Find Tripadvisor traveller reviews of <strong>San</strong> <st...

--- Result 3 ---
Title: Souvla | Greek Restaurants in California
URL: https://www.souvla.com/
Description: Souvla has locations <strong>in</strong> <strong>San</strong> <strong>Francisco</strong>&#x27;s Hayes Valley, NoPa, Mission, Marina and Dogpatch neighborhoods. Now open in Marin County! Our <strong>Re...


### Understanding the Response Structure

The Brave Search API response contains a lot of data. Let's create a helper function to extract the most relevant information from the search results that we'll need for our RAG application.

In [35]:
def extract_search_info(search_results, max_results=5):
    """
    Extract and format the most relevant information from the search results
    
    Args:
        search_results: JSON response from the Brave Search API
        max_results: Maximum number of results to extract (default: 5)
    
    Returns:
        str: Formatted string with the extracted information
    """
    if not search_results or 'web' not in search_results:
        return "No search results found."
    
    extracted_info = []
    
    # Get basic search info
    query = search_results.get('query', {}).get('query', 'Unknown query')
    extracted_info.append(f"Search query: {query}")
    extracted_info.append(f"Search date: {search_results.get('query', {}).get('timestamp', 'Unknown')}")
    
    # Extract relevant information from each result
    results = search_results.get('web', {}).get('results', [])
    for i, result in enumerate(results[:max_results]):
        if i >= max_results:
            break
            
        title = result.get('title', 'No title')
        url = result.get('url', 'No URL')
        description = result.get('description', 'No description')
        
        extracted_info.append(f"\n[{i+1}] {title}")
        extracted_info.append(f"URL: {url}")
        extracted_info.append(f"Summary: {description}")
    
    return "\n".join(extracted_info)

# Extract and display the information from our previous search
if results:
    formatted_info = extract_search_info(results)
    print(formatted_info)

Search query: Unknown query
Search date: Unknown

[1] Kokkari Estiatorio Home Page - KOKKARI
URL: https://kokkari.com/
Summary: Philoxenia Philoxenia The art of making a stranger a friend In the heart of the <strong>San</strong> <strong>Francisco</strong> Financial District you will find

[2] THE 10 BEST Greek Restaurants in San Francisco (Updated 2025)
URL: https://www.tripadvisor.com/Restaurants-g60713-c23-San_Francisco_California.html
Summary: Best <strong>Greek</strong> <strong>Restaurants</strong> <strong>in</strong> <strong>San</strong> <strong>Francisco</strong>, California: Find Tripadvisor traveller reviews of <strong>San</strong> <strong>Francisco</strong> <strong>Greek</strong> <strong>restaurants</strong> and search by price, location, and more.

[3] Souvla | Greek Restaurants in California
URL: https://www.souvla.com/
Summary: Souvla has locations <strong>in</strong> <strong>San</strong> <strong>Francisco</strong>&#x27;s Hayes Valley, NoPa, Mission, Marina and Dogpatch nei

## 3. Integrating Web Search with LLMs

Now that we understand how to use the Brave Search API, let's integrate it with OpenAI's chat completions to create an agent that can search the web and use that information to answer questions.

### 3.1 Simple Chat Completion

First, let's see how we can use OpenAI's chat completion API to answer questions without any web search capabilities. This will serve as our baseline.

In [45]:

def basic_chat_completion(query):

    try:
        response = client.chat.completions.create(
            model=MODEL, 
            messages=[
                {"role": "system", "content": "You are a helpful assistant"},
                {"role": "user", "content": query}
            ],
            temperature=0, # Controls randomness in the response
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

In [46]:
# Example: Ask a question that might benefit from current information
query = "What were the major tech news headlines today?"
response = basic_chat_completion(query)
print(f"Query: {query}")
print(f"\nResponse without web search:\n{response}")

Query: What were the major tech news headlines today?

Response without web search:
I'm unable to provide real-time news updates or current headlines as my training only includes information up until October 2023. However, you can check reliable news websites, tech blogs, or news aggregators for the latest tech news headlines. If you have any specific topics or events in mind, I can provide background information or context based on my training data.


### 3.2 Manual RAG with Web Search

Now, let's create a simple RAG (Retrieval Augmented Generation) system that first searches the web and then uses the search results as context for the LLM.

In [ ]:
def rag_web_search(query):
    """
    Implement a basic RAG system using web search and chat completion
    
    Args:
        query: User's question
        
    Returns:
        tuple: (Model's response, Search results used as context)
    """
    # 1. Perform web search
    search_results = brave_search(query)
    
    # 2. Extract relevant information
    if not search_results:
        return "Couldn't perform web search.", ""
    
    context = extract_search_info(search_results, max_results=5) # this is the context we will use as part of the prompt
    
    # 3. Create prompt with search results as context
    prompt = f"""
    Based on the following information from a web search, please answer the question.

    QUESTION: 
    {query}
    
    SEARCH RESULTS:
    {context}
    
    Please provide a comprehensive answer based on the search results. If the search results don't contain 
    relevant information to answer the question, please state that and provide your best response based on 
    your knowledge.
    """
    
    # 4. Get response from LLM
    try:
        response = client.chat.completions.create(
            model=MODEL, 
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions based on web search results."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
        )
        response_message = response.choices[0].message.content

        return response_message, context
    except Exception as e:
        return f"Error: {str(e)}", context

In [54]:
print( """
    Based on the following information from a web search, please answer the question.
    
    QUESTION: 
    {query}

    SEARCH RESULTS:
    {context}
    
    Please provide a comprehensive answer based on the search results. If the search results don't contain 
    relevant information to answer the question, please state that and provide your best response based on 
    your knowledge.
    """.format(context=context, query=query))


    Based on the following information from a web search, please answer the question.

    QUESTION: 
    What were the major tech news headlines today?

    SEARCH RESULTS:
    Search query: Unknown query
Search date: Unknown

[1] Reuters Tech News | Today's Latest Technology News | Reuters
URL: https://www.reuters.com/technology/
Summary: Find latest technology <strong>news</strong> from every corner of the globe at Reuters.com, your online source for breaking international <strong>news</strong> coverage.

[2] Tech | CNN Business
URL: https://www.cnn.com/business/tech
Summary: View the latest technology <strong>headlines</strong>, gadget and smartphone trends, and insights from <strong>tech</strong> industry leaders.

[3] TechNewsWorld - Technology News and Information
URL: https://www.technewsworld.com
Summary: <strong>Tech</strong> <strong>news</strong>, reviews and analysis of computing, enterprise IT, cybersecurity, mobile technology, cloud computing, <strong>tech</strong> indus

In [49]:
# Example: Ask the same question but with web search
query = "What were the major tech news headlines today?"
response, context = rag_web_search(query)

In [55]:
print("Query:", query)
print("\nWeb Search Results Used as Context:")
print("-" * 80)
print(context)
print("-" * 80)

Query: What were the major tech news headlines today?

Web Search Results Used as Context:
--------------------------------------------------------------------------------
Search query: Unknown query
Search date: Unknown

[1] Reuters Tech News | Today's Latest Technology News | Reuters
URL: https://www.reuters.com/technology/
Summary: Find latest technology <strong>news</strong> from every corner of the globe at Reuters.com, your online source for breaking international <strong>news</strong> coverage.

[2] Tech | CNN Business
URL: https://www.cnn.com/business/tech
Summary: View the latest technology <strong>headlines</strong>, gadget and smartphone trends, and insights from <strong>tech</strong> industry leaders.

[3] TechNewsWorld - Technology News and Information
URL: https://www.technewsworld.com
Summary: <strong>Tech</strong> <strong>news</strong>, reviews and analysis of computing, enterprise IT, cybersecurity, mobile technology, cloud computing, <strong>tech</strong> industry tre

In [56]:
# LLM Output
print("\nResponse with web search:")
print(response)


Response with web search:
The search results provided do not contain specific information about today's major tech news headlines. They include links to various technology news sources such as Reuters, CNN Business, TechNewsWorld, CNBC, and WIRED, but none of them list current headlines or articles.

To find the latest major tech news headlines, I recommend visiting one of the following websites directly:

1. **Reuters Technology**: [Reuters Tech News](https://www.reuters.com/technology/)
2. **CNN Business Tech**: [CNN Business Tech](https://www.cnn.com/business/tech)
3. **TechNewsWorld**: [TechNewsWorld](https://www.technewsworld.com)
4. **CNBC Technology**: [CNBC Technology](https://www.cnbc.com/technology/)
5. **WIRED**: [WIRED](https://www.wired.com/)

These sources regularly update their content with the latest news in technology, including major headlines, trends, and insights from industry leaders.


## 4. Using Tools and Functions with OpenAI

The manual RAG approach works, but it has some limitations. The model doesn't get to decide when it needs to search for information. Let's use OpenAI's function calling capability to create a more dynamic agent that can choose when to search the web.

In [57]:
# Define the search tool
tool_web_search = {
    "type": "function",
    "function": {
        "name": "web_search",
        "description": "Search the web for current information on a given topic",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query to look up on the web"
                }
            },
            "required": ["query"]
        }
    }
}

# Define the actual function that will be called
def web_search(query):
    """
    Search the web using Brave Search API
    
    Args:
        query: The search query
        
    Returns:
        str: Formatted search results
    """
    results = brave_search(query)
    if results:
        return extract_search_info(results)
    else:
        return "No search results found."

In [58]:
prompt_agent_web_search = """
        You are a helpful assistant with access to web search. 
        When a user asks a question that might benefit from current information from the web, 
        use the search_web function to look up relevant information before answering. 
        For questions that don't require current information, answer directly using your knowledge.
        
        When using search results:
        1. Be explicit that you're using web search data
        2. Cite the sources mentioned in the search results
        3. Synthesize information from multiple results when possible
        """

In [73]:
query = "What is the bitcoin price today?"

def agent_web_search(query):
    """
    Use the web search tool to find current information on a given topic.
    
    Args:
        query: The search query to look up on the web
        
    Returns:
        str: The response from the model after performing the web search
    """
    # Create the messages for the chat completion
    messages = [{"role": "system", "content": prompt_agent_web_search},
                {"role": "user", "content": query}
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=[tool_web_search],
        tool_choice="auto",  # let the model decide
        temperature=0,
    )
    response_message = response
    return response_message

In [74]:
agent_output = agent_web_search(query)

In [75]:
# when a tool is called, the response content will be None
print(agent_output.choices[0].message.content)

None


In [76]:
# to get the tool call, we can access it like this
tool_call = agent_output.choices[0].message.tool_calls[0]
print(f"Tool called: {tool_call.function.name}")

Tool called: web_search


In [77]:
# We can also access the arguments passed to the tool
tool_args = tool_call.function.arguments
print(f"Arguments passed to the tool: {tool_args}")

Arguments passed to the tool: {"query":"current Bitcoin price"}


In [78]:
type(tool_args)

str

In [70]:
# parse the arguments to the web_search function

tool_args_dict = json.loads(tool_args) # convert JSON string to dictionary
search_query = tool_args_dict.get("query", "") # get the query from the arguments
search_results = web_search(search_query)

In [72]:
print(search_results)

Search query: Unknown query
Search date: Unknown

[1] Bitcoin price today, BTC to USD live price, marketcap and chart | CoinMarketCap
URL: https://coinmarketcap.com/currencies/bitcoin/
Summary: The live Bitcoin price today is <strong>$108,650 USD</strong> with a 24-hour trading volume of $42,288,097,430 USD. We update our BTC to USD price in real-time. Bitcoin is up 0.80% in the last 24 hours. The current CoinMarketCap ranking is #1, with a live market cap of $2,161,011,875,679 USD.

[2] Bitcoin (BTC) Price | BTC to USD Price and Live Chart
URL: https://www.coindesk.com/price/bitcoin
Summary: The price of Bitcoin (BTC) is <strong>$108,202.05</strong> today as of Jul 8, 2025, 2:11 am EDT, with a 24-hour trading volume of $14.06B.

[3] Bitcoin Price, BTC Price, Live Charts, and Marketcap: btc, bitcoin price usd, btc price usd
URL: https://www.coinbase.com/price/bitcoin
Summary: We update our Bitcoin to USD currency in real-time. Get the live price of Bitcoin on Coinbase. ... The current 

In [80]:
prompt_agent_final_response = """
Based on the web search results, please provide a final response to the user's query.
## USER QUERY:
{user_query}

## SEARCH RESULTS:
{search_results}
"""

def agent_final_response(query, search_results):
    """
    Generate a final response based on the user's query and search results.
    
    Args:
        quer: The user's query
    Returns:
        str: The final response generated by the model
    """
    # Create the messages for the chat completion
    messages = [{"role": "system", "content": prompt_agent_final_response.format(user_query=query, search_results=search_results)},]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0,
    )
    response_message = response.choices[0].message.content
    return response_message

In [ ]:
agent_final_response_output = agent_final_response(query, search_results)
print(agent_final_response_output)

As of today, the price of Bitcoin (BTC) is approximately **$108,650 USD** according to CoinMarketCap. Other sources report slightly different prices, with CoinDesk listing it at **$108,202.05** and Yahoo Finance at **$108,491.68**. The price can vary slightly depending on the source, but it is generally in the range of **$108,200 to $108,650 USD**. Please check a reliable financial news website or cryptocurrency exchange for the most current price.


## 5. Advanced Prompt Engineering for Web-Enabled Agents

The system prompt is crucial for creating effective web-enabled agents. Let's explore how we can improve our agent by enhancing the prompt with specific instructions on:

1. When to search vs. use existing knowledge
2. How to synthesize information from multiple sources
3. How to handle contradictory information
4. How to properly attribute sources

In [86]:
improved_system_prompt = """
You are an advanced web-enabled research assistant with access to real-time information through web search.

WHEN TO SEARCH:
- Always search for current events, news, prices, weather, sports scores, or other time-sensitive information
- Search for specific facts, statistics, or data that may have changed or been updated since your training
- Search when asked about recent products, publications, or developments
- Use your existing knowledge for general concepts, historical facts, or timeless information

SEARCH PROCESS:
1. First consider if the query requires current or specific information
2. If so, formulate a clear, concise search query focused on the key information needed
3. Analyze the search results thoroughly before responding
4. If the search doesn't return useful information, try reformulating your query or state that current information isn't available

SYNTHESIS AND ATTRIBUTION:
- Synthesize information from multiple search results when available
- Explicitly cite sources with phrases like "According to [source]..."
- When sources conflict, acknowledge the differences and explain possible reasons
- Always make clear which parts of your response come from web search vs. your knowledge
- Use numbered references when citing multiple sources (e.g., [1], [2])

RESPONSE FORMAT:
- Begin with a direct answer to the query
- Support with evidence from search results or your knowledge
- Provide balanced viewpoints when the topic is subjective
- End with any relevant caveats, limitations, or suggestions for further information
"""

In [100]:
def agent_advanced_web_search(query):
    """
    An improved web-enabled agent with better prompting
    
    Args:
        query: User's question
        
    Returns:
        tuple: (Agent's final response, Conversation history with search results)
    """
    
    # Strict prompt that forces search-first behavior
    strict_prompt = """
You are a web-enabled research assistant. For ANY query about current, latest, or recent information, you MUST search first before providing any response.

CRITICAL RULES:
1. When asked about "latest", "current", "recent", "today's", or comparative product information, ALWAYS use the web_search tool FIRST
2. Do NOT provide any response content until you have search results
3. Base your final answer ONLY on the search results provided
4. If search results are insufficient, state exactly what information is missing
5. Always cite your sources from the search results

Your response should be: [search first, then respond based only on search results]
"""
    
    messages = [
        {"role": "system", "content": strict_prompt},
        {"role": "user", "content": query}
    ]
    
    conversation_history = [f"User: {query}"]
    
    # Check if this is a query that requires current information
    current_info_keywords = ['latest', 'current', 'recent', 'today', 'now', 'compare', '2024', '2025']
    needs_current_info = any(keyword in query.lower() for keyword in current_info_keywords)
    
    if needs_current_info:
        # Force search for current information
        conversation_history.append("[System: Query requires current information - forcing web search]")
        
        # Directly perform the search without asking the model first
        search_results = web_search(query)
        conversation_history.append(f"[Searched for: {query}]")
        conversation_history.append(f"Search results: {search_results}")
        
        # Now ask the model to respond based only on search results
        search_based_messages = [
            {"role": "system", "content": "You are a helpful assistant. Provide a comprehensive answer based ONLY on the search results provided. Do not use any information from your training data."},
            {"role": "user", "content": f"Question: {query}\n\nSearch Results:\n{search_results}\n\nPlease answer the question using ONLY the information from these search results. If the search results don't contain enough information, explicitly state what's missing."}
        ]
        
        response = client.chat.completions.create(
            model=MODEL,
            messages=search_based_messages,
            temperature=0,
        )
        
        final_response = response.choices[0].message.content
        conversation_history.append(f"Assistant: {final_response}")
        
    else:
        # For non-current queries, let the model decide
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=[tool_web_search],
            tool_choice="auto",
            temperature=0,
        )
        response_message = response.choices[0].message
        
        # Handle tool calls if any
        if hasattr(response_message, 'tool_calls') and response_message.tool_calls:
            for tool_call in response_message.tool_calls:
                function_args = json.loads(tool_call.function.arguments)
                search_query = function_args.get("query")
                
                search_results = web_search(search_query)
                conversation_history.append(f"[Searched for: {search_query}]")
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": "web_search",
                    "content": search_results,
                })
            
            # Get final response with search results
            second_response = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                temperature=0,
            )
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
        
        conversation_history.append(f"Assistant: {final_response}")
    
    return final_response, conversation_history

In [101]:
# Example query that benefits from the improved prompt
complex_query = "Compare the latest iPhone model with the latest Samsung Galaxy. What are the main differences in features, price, and reviews?"

response, history = agent_advanced_web_search(complex_query)

print("Query:", complex_query)
print("\nFull Conversation History:")
for entry in history:
    print("\n" + entry)

Query: Compare the latest iPhone model with the latest Samsung Galaxy. What are the main differences in features, price, and reviews?

Full Conversation History:

User: Compare the latest iPhone model with the latest Samsung Galaxy. What are the main differences in features, price, and reviews?

[System: Query requires current information - forcing web search]

[Searched for: Compare the latest iPhone model with the latest Samsung Galaxy. What are the main differences in features, price, and reviews?]

Search results: Search query: Unknown query
Search date: Unknown

[1] iPhone vs Samsung: The ultimate head-to-head - Android Authority
URL: https://www.androidauthority.com/iphone-vs-samsung-3266172/
Summary: The Face of Android goes up against the only game in town for iOS. How does the Apple <strong>iPhone</strong> <strong>compare</strong> to <strong>the</strong> <strong>Samsung</strong> <strong>Galaxy</strong> today?

[2] iPhone vs Samsung: Latest Phones comparison in 2025
URL: https:

In [104]:
print(response)

Based on the search results, here is a comparison of the latest iPhone model (iPhone 16) and the latest Samsung Galaxy model (Galaxy S25):

### Features
1. **Build Quality**: The iPhone 16, particularly the Pro models, is noted for its superior build quality, utilizing materials like stainless steel or titanium. In contrast, the Galaxy S25's build quality is not specifically mentioned, but it is generally known for its premium materials as well.

2. **Design**: The search results do not provide specific details on the design differences between the iPhone 16 and Galaxy S25. However, it is common for both brands to have distinct design philosophies, with Apple favoring a more minimalist aesthetic and Samsung often incorporating more vibrant colors and curved displays.

3. **Camera**: The search results do not provide direct comparisons of camera specifications or performance between the two models. Typically, both brands offer advanced camera systems, but specific features like zoom cap

## 6. Multi-Step Research with Chain-of-Thought

For complex research questions, a single search might not be sufficient. Let's implement a multi-step research approach that allows the agent to perform multiple searches, each informed by the results of previous searches.

In [107]:
def multi_step_research_agent(query, max_steps=5):
    """
    Create an agent that can perform multi-step research with chain-of-thought reasoning
    
    Args:
        query: User's research question
        max_steps: Maximum number of research steps to perform
        
    Returns:
        tuple: (Agent's final response, Research log with all steps and searches)
    """
    research_log = [f"RESEARCH QUESTION: {query}"]
    
    # Enhanced chain-of-thought system prompt
    cot_system_prompt = """
You are an expert research assistant that performs systematic, multi-step research using web search.

RESEARCH METHODOLOGY:
1. ANALYZE: Break down complex questions into specific sub-questions
2. PLAN: Determine what information you need to search for
3. SEARCH: Perform targeted searches for each piece of information
4. EVALUATE: Assess the quality and relevance of search results
5. SYNTHESIZE: Combine findings into a comprehensive answer

CHAIN-OF-THOUGHT PROCESS:
For each step, you must:
- State your current research objective
- Explain your reasoning for the next search
- Analyze what you learned from search results
- Identify gaps that need further investigation
- Build upon previous findings

SEARCH STRATEGY:
- Start with broad searches, then narrow down to specifics
- Search for different perspectives and sources
- Look for recent data, studies, and expert opinions
- Cross-reference conflicting information

QUALITY STANDARDS:
- Only use information from search results
- Cite sources explicitly
- Acknowledge limitations in available data
- Present balanced viewpoints when applicable
"""
    
    messages = [
        {"role": "system", "content": cot_system_prompt},
        {"role": "user", "content": f"Research question: {query}\n\nPlease start by analyzing this question and planning your research approach. Break it down into sub-questions and explain your strategy."}
    ]
    
    accumulated_knowledge = []
    search_count = 0
    
    for step in range(max_steps):
        research_log.append(f"\n--- RESEARCH STEP {step+1} ---")
        
        # Get next action from model
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=[tool_web_search],
            tool_choice="auto",
            temperature=0.1,  # Slight temperature for more varied search strategies
        )
        response_message = response.choices[0].message
        
        # Add response to conversation
        messages.append(response_message.model_dump())
        
        # Log the model's reasoning
        if response_message.content:
            research_log.append(f"THINKING: {response_message.content}")
        
        # Handle tool calls (searches)
        if hasattr(response_message, 'tool_calls') and response_message.tool_calls:
            for tool_call in response_message.tool_calls:
                function_args = json.loads(tool_call.function.arguments)
                search_query = function_args.get("query")
                search_count += 1
                
                research_log.append(f"SEARCH #{search_count}: {search_query}")
                
                # Execute search
                search_results = web_search(search_query)  # Get more results for better coverage
                research_log.append(f"RESULTS: {search_results[:500]}...")  # Truncate for readability
                
                # Store search results for synthesis
                accumulated_knowledge.append({
                    "query": search_query,
                    "results": search_results
                })
                
                # Send results back to model
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": "web_search",
                    "content": search_results,
                })
        
        # Check if we should continue researching
        if not (hasattr(response_message, 'tool_calls') and response_message.tool_calls):
            # Model didn't request more searches
            continue_prompt = f"Do you need to search for more information to thoroughly answer: '{query}'? If yes, specify what additional information you need and search for it. If no, prepare for final synthesis."
            
            messages.append({"role": "user", "content": continue_prompt})
            
            continue_response = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                tools=[tool_web_search],
                tool_choice="auto",
                temperature=0,
            )
            
            continue_message = continue_response.choices[0].message
            messages.append(continue_message.model_dump())
            
            if continue_message.content:
                research_log.append(f"CONTINUATION DECISION: {continue_message.content}")
            
            # If no more tool calls, we're done with research
            if not (hasattr(continue_message, 'tool_calls') and continue_message.tool_calls):
                research_log.append("Research phase complete - moving to synthesis.")
                break
            else:
                # Handle additional searches
                for tool_call in continue_message.tool_calls:
                    function_args = json.loads(tool_call.function.arguments)
                    search_query = function_args.get("query")
                    search_count += 1
                    
                    research_log.append(f"ADDITIONAL SEARCH #{search_count}: {search_query}")
                    
                    search_results = web_search(search_query)
                    research_log.append(f"RESULTS: {search_results[:500]}...")
                    
                    accumulated_knowledge.append({
                        "query": search_query,
                        "results": search_results
                    })
                    
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": "web_search",
                        "content": search_results,
                    })
    
    # Final synthesis phase
    research_log.append("\n--- FINAL SYNTHESIS ---")
    
    # Create comprehensive context from all searches
    synthesis_context = f"Original question: {query}\n\n"
    synthesis_context += "All research findings:\n"
    for i, knowledge in enumerate(accumulated_knowledge, 1):
        synthesis_context += f"\nSearch {i}: {knowledge['query']}\nResults: {knowledge['results']}\n"
    
    synthesis_prompt = f"""
Based on all the research you've conducted, provide a comprehensive, well-structured answer to the original question.

{synthesis_context}

Requirements for your final answer:
1. Provide a clear, direct response to the question
2. Structure your answer logically with clear sections
3. Support all claims with evidence from your search results
4. Cite specific sources where possible
5. Acknowledge any limitations or areas where information was insufficient
6. Present a balanced view when multiple perspectives exist
7. Conclude with key takeaways or recommendations

Question to answer: {query}
"""
    
    messages.append({"role": "user", "content": synthesis_prompt})
    
    final_response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0,
    )
    
    final_answer = final_response.choices[0].message.content
    research_log.append(f"FINAL ANSWER:\n{final_answer}")
    
    # Add summary statistics
    research_log.append(f"\n--- RESEARCH SUMMARY ---")
    research_log.append(f"Total searches performed: {search_count}")
    research_log.append(f"Research steps completed: {step + 1}")
    
    return final_answer, research_log

In [108]:
# Example complex research question
research_question = "What are the environmental impacts of electric vehicles compared to conventional cars, considering their entire lifecycle?"

final_answer, research_log = multi_step_research_agent(research_question)

print("Research Question:", research_question)
print("\nResearch Process and Results:")
for entry in research_log:
    print("\n" + entry)

Research Question: What are the environmental impacts of electric vehicles compared to conventional cars, considering their entire lifecycle?

Research Process and Results:

RESEARCH QUESTION: What are the environmental impacts of electric vehicles compared to conventional cars, considering their entire lifecycle?


--- RESEARCH STEP 1 ---

THINKING: ### Research Objective
The objective is to understand the environmental impacts of electric vehicles (EVs) compared to conventional internal combustion engine (ICE) vehicles throughout their entire lifecycle, including production, usage, and disposal.

### Sub-Questions
1. **Lifecycle Assessment**: What are the stages of the lifecycle for both electric vehicles and conventional cars?
2. **Manufacturing Impact**: How does the manufacturing process of EVs compare to that of conventional cars in terms of resource extraction, energy consumption, and emissions?
3. **Operational Impact**: What are the emissions and energy consumption during the 

## 7. Conclusion and Best Practices

In this lesson, we've learned how to build AI agents that can access the web in real-time using the Brave Search API. Here's a summary of what we've covered:

1. **Basic web search with Brave Search API**
   - Making API calls to retrieve search results
   - Parsing and extracting relevant information

2. **Simple RAG approach**
   - Manually fetching web data and using it as context
   - Comparing results with and without web access

3. **Function-calling for dynamic web access**
   - Using OpenAI's function calling to let the model decide when to search
   - Creating tools for the model to use

4. **Advanced prompt engineering**
   - Crafting effective system prompts
   - Guiding the model on when to search vs. use existing knowledge

5. **Multi-step research with chain-of-thought**
   - Breaking down complex queries
   - Building cumulative knowledge through multiple searches

### Best Practices

When building web-enabled agents, consider these best practices:

1. **Be transparent** - Make it clear when information comes from web search
2. **Cite sources** - Always attribute information to its source
3. **Be selective with searches** - Don't search unnecessarily
4. **Handle uncertainty** - Acknowledge when information might be incomplete or conflicting
5. **Consider freshness** - Prioritize recent information for time-sensitive queries
6. **Multi-step research** - Break down complex questions into sub-questions
7. **Validate information** - Cross-check critical facts with multiple sources

### Exercises

Try implementing the following on your own:

1. Add a tool for image search and incorporate image descriptions into responses
2. Implement a "fact-checking" function that verifies information against multiple sources
3. Create a specialized agent for a specific domain (e.g., science news, finance, etc.)
4. Add memory to your agent so it can remember previous searches and build on them